<h1><center>CMPE 462 - Project 2<br>Implementing an SVM Classifier<br>Due: May 18, 2020, 23:59</center></h1>

* **Student ID1:** 2016400186
* **Student ID2:** 2016400078
* **Student ID3:** 2016400003

## Overview

In this project, you are going to implement SVM. For this purpose, a data set (data.mat) is given to you. You can load the mat dataset into Python using the function `loadmat` in `Scipy.io`. When you load the data, you will obtain a dictionary object, where `X` stores the data matrix and `Y` stores the labels. You can use the first 150 samples for training and the rest for testing. In this project, you will use the software package [`LIBSVM`](http://www.csie.ntu.edu.tw/~cjlin/libsvm/) to implement SVM. Note that `LIBSVM` has a [`Python interface`](https://github.com/cjlin1/libsvm/tree/master/python), so you can call the SVM functions in Python. 

In [6]:
import scipy.io as sp
import numpy as np
import pandas as pd
data = sp.loadmat("data.mat")
X = data.get("X")
y = data.get("Y")
X_train, y_train = X[:150], y[:150].flatten()
X_test, y_test = X[150:], y[150:].flatten()

## Task 1 - 30 pts

Train a hard margin linear SVM and report both train and test classification accuracy.

In [11]:
from libsvm.svmutil import *
prob = svm_problem(y_train, X_train)
params = svm_parameter("-t 0 -c 1e8")
model = svm_train(prob, params)
p_label, p_acc, p_val = svm_predict(y_train, X_train, model)
p_label, p_acc, p_val = svm_predict(y_test, X_test, model)

Accuracy = 74.6667% (112/150) (classification)
Accuracy = 77.5% (93/120) (classification)


## Task 2 - 40 pts

Train soft margin SVM for different values of the parameter $C$, and with different kernel functions. Systematically report your results. For instance, report the performances of different kernels for a fixed $C$, then report the performance for different $C$ values for a fixed kernel, and so on.

In [12]:
kernel_functions = ["linear", "polynomial", "rbf", "sigmoid"]                          # kernel function list
sv_number = [[""],[""], [""], [""]]
df = {"linear":[""], "sv_l":[""],  "polynomial":[""], "sv_p":[""], "rbf":[""], "sv_r":[""], "sigmoid":[""], "sv_s":[""] } # dataframe construction to visualize
c_values = ["C-Values"]
for kernel_function in range(len(kernel_functions)):        # for all kernel function given above
    for c in range(1,1001,15):                              # for some c values
        parameters = svm_parameter("-t {} -c {}".format(kernel_function, c/100))           # parameterize
        model = svm_train(prob, parameters)                                                # train
        p_label, p_acc, p_val = svm_predict(y_test, X_test, model, "-q")                   # accuracy
        df[kernel_functions[kernel_function]].append("{:.4f}".format(p_acc[0]))            # append to df
        sv_number[kernel_function].append(len(model.get_SV()))
        if c/100 not in c_values:
            c_values.append(c/100)
[df["sv_l"], df["sv_p"], df["sv_r"], df["sv_s"]]  = sv_number
pd.DataFrame(df, c_values)

,linear,sv_l,polynomial,sv_p,rbf,sv_r,sigmoid,sv_s
C-Values,,,,,,,,
0.01,84.1667,118,58.3333,141,58.3333,140,58.3333,140
0.16,85.0000,69,58.3333,141,84.1667,117,84.1667,115
0.31,83.3333,67,84.1667,141,84.1667,103,85.0000,98
0.46,85.0000,63,79.1667,134,83.3333,92,84.1667,89
...,...,...,...,...,...,...,...,...
9.31,81.6667,51,80.8333,85,77.5000,74,81.6667,56
9.46,81.6667,51,80.8333,85,77.5000,74,81.6667,56
9.61,81.6667,51,80.8333,85,77.5000,74,80.8333,56
9.76,81.6667,51,80.8333,85,77.5000,74,81.6667,57


## Task 3 - 15 pts

Please report how the number of support vectors changes as the value of $C$ increases (while all other parameters remain the same). Discuss whether your observations match the theory.

* In theory, as the value of C increses, the number of support vectors decrease, and accuracy of the model up to a c-value. 
* We know that if C is very high, model can turn into hard margin svm, if C is very low, model can ignore the error term and accuracy can be low. 
* The decrease in support vectors means that there are less vector that are supporting the margin. This can be because of we apply some regularization called soft margin, we ignore some support vectors and increase the fatness. Therefore, the number of supporting vectors decrease. 
* The decrease in support vector can be seen clearly from previous figure, which is said also in theory, so correlation make our work right.
  * We see that, with polynomial, rbf and sigmoid we see many error with small C value. As C increases, they reach a top point and then start to decrease again. This is because it went to hard margin svm and lost some regularization

## Task 4 - 15 pts

Please investigate the changes in the hyperplane when you remove one of the support vectors, vs., one data point that is not a support vector.

In [13]:
def getHyperPlane(sv_dict, col, model):
    sv_hyperplane = np.zeros((len(sv_dict),col))     # hyperplane matrix, initiated 
    j = 0 
    for each in sv_dict:                             # for each support vector
        for i in range(col):                         # for each attribute
            if i+1 in each:                           # if (i+1) labeled element in support vector dictionary, we save
                sv_hyperplane[j][i] = each.get(i+1)
        j+=1
    return (model.get_sv_coef() * sv_hyperplane)        # by multiplying w * H we obtain, real hyperplane Matrix

row, col = np.shape(X_train)

model = svm_train(prob, "-t 0")                          # training model using linear kernel
sv_hyperplane = getHyperPlane(model.get_SV(), col, model)               

#####################  w of default

sv_to_delete_dict = model.get_SV()[1]                         # we choose 1st support vector to delete from our data
sv_to_delete_list = np.zeros(col)
for i in range(col):                                          # for each of attributes, we construct sv list from sv dict.
    if i+1 in sv_to_delete_dict:
        sv_to_delete_list[i] = (sv_to_delete_dict.get(i+1))
ind = np.where((X_train == sv_to_delete_list).all(axis=1))[0][0]    # finding indice to delete from dataset

X_train_sv_deleted = np.delete(X_train, ind, axis = 0)              # deleting sv from datased
y_train_sv_deleted = np.delete(y_train, ind)                        # deleting corresponding label
prob_sv_deleted = svm_problem(y_train_sv_deleted, X_train_sv_deleted)        # constructing svm problem with new dataset
model_sv_deleted = svm_train(prob_sv_deleted, "-t 0")                        # training

sv_deleted_hyperplane = getHyperPlane(model_sv_deleted.get_SV(), col, model_sv_deleted)      

######################### w of sv deleted

indice = 0                                                 # we will search for a non-sv data point to delete
for i in range(row):                                       # for each data point
    normal_point = X_train[i,:]          
    sv_check = {}
    for i in range(col):                                   # construct its sparse dictionary
        if normal_point[i] != 0:
            sv_check[i+1] = normal_point[i]

    if sv_check not in model.get_SV():                     # if it is not in support vectors, we found one
        indice = i
        break

X_train_nm_deleted = np.delete(X_train, indice, axis = 0)   # delete non - sv
y_train_nm_deleted = np.delete(y_train, indice)             # delete non - sv
prob_nm_deleted = svm_problem(y_train_nm_deleted, X_train_nm_deleted)         # new problem
model_nm_deleted = svm_train(prob_nm_deleted, "-t 0")                         # new training

nm_deleted_hyperplane = getHyperPlane(model_nm_deleted.get_SV(), col, model_nm_deleted)        # hyperplane

############################ w or data point deleted

print(np.shape(nm_deleted_hyperplane))
print(np.shape(sv_deleted_hyperplane))
print(np.shape(sv_hyperplane))

(58, 13)
(55, 13)
(58, 13)


* Number of support vectors decreased not only by the deleted one, but 3.

### Bonus Task - 10 pts

Use Python and [CVXOPT](http://cvxopt.org) QP solver to implement the hard margin SVM. 

In [14]:
#Code to run cvxopt is mainly taken from https://xavierbourretsicotte.github.io/SVM_implementation.html 
#where it is mainly taken from doccumentation
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers

#Initializing values and computing H. Note the 1. to force to float type
row, col = X_train.shape
y = y_train.reshape(row,1) * 1.
X_dash = y * X_train
H = X_dash @ X_dash.T * 1.

#Converting into cvxopt format
P = cvxopt_matrix(H)
q = cvxopt_matrix(-np.ones((row, 1)))
G = cvxopt_matrix(-np.eye(row))
h = cvxopt_matrix(np.zeros(row))
A = cvxopt_matrix(y.reshape(1, -1))
b = cvxopt_matrix(np.zeros(1))

#Setting solver parameters (change default to decrease tolerance) 
cvxopt_solvers.options['show_progress'] = False
cvxopt_solvers.options['abstol'] = 1e-10
cvxopt_solvers.options['reltol'] = 1e-10
cvxopt_solvers.options['feastol'] = 1e-10

#Run solver
sol = cvxopt_solvers.qp(P, q, G, h, A, b)
alphas = np.array(sol['x'])

ModuleNotFoundError: No module named 'cvxopt'

In [15]:
#w parameter in vectorized form
w = ((y * alphas).T @ X_toy).reshape(col,1)

#Selecting the set of indices S corresponding to non zero parameters
S = (alphas > 1e-4).flatten()
b = (y[S] - np.dot(X_toy[S], w))[0]

print('w = ', w)
print('b = ', b)

NameError: name 'alphas' is not defined

In [7]:
print ( np.sum(np.sign( X_toy @ w + b) == y) / len(y)) 